In [12]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import markdown
import re
import textwrap
from IPython.display import Markdown

Markdown as Mentioned in Google API DOCS 

In [13]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

Markdown as by html parser

In [14]:
def toString(input_string):
    # print(input_string)
    html = markdown.markdown(input_string)
    text = BeautifulSoup(html, features='html.parser')
    text = text.get_text().strip().replace('\n',' ')
    text = text.strip().replace('*','')
    # text = re.sub(r'(?<!^)(?=[A-Z])', ' ', text)
    # res = re.sub(' +', ' ', text)
    return text

Converting each test cases to Required Format

In [15]:
def toDataFrame(res,userstories):
    
    # Split the string into individual test cases
    test_cases = res.split("Test Case")[1:]

    # Initialize lists to store data
    descriptions = []
    inputs = []
    expected_outputs = []

    # Extract information for each test case
    for test_case in test_cases:
        parts = test_case.split("Description")
        # print(parts)
        if "Description" in test_case:
            description = parts[1].split("Input:")[0].strip()
            input_part = parts[1].split("Input:")[1]
            input_data = input_part.split("Expected Output:")[0].strip()
            expected_output = input_part.split("Expected Output:")[1].strip()
            
            descriptions.append(description)
            inputs.append(input_data)
            expected_outputs.append(expected_output)

    # Create a dataframe
    data = {
        "User Story":userstories,
        "Description": descriptions,
        "Input": inputs,
        "Expected Output": expected_outputs
    }
    return data

In [60]:
excel_file = 'train.xlsx'
df = pd.read_excel(excel_file)

In [61]:
data = pd.DataFrame(columns=["User Story","Description","Input","Expected Output"])

In [62]:
input_string = df['response'][702]
user_story = df['userstories'][702]
input_string = toString(input_string)

print(toString(input_string))
data[len(data)] = toDataFrame(input_string,user_story)

Test Case 1: Description Provide a valid dataset with all required metadata fields populated. Input:  Dataset in CSV or Excel format Metadata file in JSON or XML format with the following fields: Dataset title Dataset description Dataset author Dataset keywords Dataset publication date Dataset license    Expected Output:  Dataset and metadata successfully ingested into the repository.  Test Case 2: Description Provide a dataset with some missing metadata fields. Input:  Dataset in CSV or Excel format Metadata file in JSON or XML format with the following fields: Dataset title Dataset description Dataset author Dataset keywords    Expected Output:  Dataset ingested into the repository with a warning about missing metadata fields. Repository staff contacts the batch submitter to obtain the missing metadata.  Test Case 3: Description Provide a dataset with invalid data values. Input:  Dataset in CSV or Excel format with invalid data values, such as empty cells or date values in the wrong 

In [63]:
df

,Unnamed: 0,role,userstories,response
0,0,Data user,I want to have the 12-19-2017 deletions proce...,> **Test Case 1**\n> * **Description:** Proces...
1,1,UI designer,"I want to redesign the Resources page, so tha...",> **Test Case 1**\n> \n> **Description:** Veri...
2,2,UI designer,I want to report to the Agencies about user t...,> **Test Case 1**\n> \n> **Description:** Veri...
3,3,UI designer,I want to move on to round 2 of DABS or FABS ...,> **Test Case 1**\n> \n> * **Description:** Ve...
4,4,UI designer,I want to move on to round 2 of Homepage edit...,> **Test Case 1**\n> **Description:** Ensure t...
...,...,...,...,...
1662,1663,admin,I want to know when Zoonibot should give an e...,> **Test Case 1:**\n> * **Description** User a...
1663,1664,admin,I want to know what Zoonibot should say to a ...,> **Test Case 1:**\n> * **Description** The vo...
1664,1665,admin,I want to group subjects by similarity.,> **Test Case 1:**\n> \n> * **Description** Gr...
1665,1666,admin,I want to recommend different projects to vol...,> **Test Case 1:**\n> \n> * **Description** Re...


In [69]:
for index, row in df.iterrows():
    
    user_story = df['userstories'][index]
    input_string = df['response'][index]
    input_string = toString(input_string)
    #data = data.concat([data,toDataFrame(input_string,user_story,data)],ignore_index = True)
    # data.loc[len(df)] = toDataFrame(input_string,user_story,data)
    # print(data)
        
    # Split the string into individual test cases
    test_cases = input_string.split("Test Case")[1:]

    # Initialize lists to store data
    descriptions = []
    inputs = []
    expected_outputs = []

    # Extract information for each test case
    for test_case in test_cases:
        parts = test_case.split("Description")
        # print(parts)
        if "Description" in test_case:
            description = parts[1].split("Input")[0].strip()
            # print(description)
            if "Input" in test_case:
                input_part = parts[1].split("Input")[1].strip()
                print(input_part)
                input_data = input_part.split("Expected Output")[0].strip()
                print(input_part)
                if "Expected Output" in test_case:
                    expected_output = input_part.split("Expected Output")[1].strip()
                    # print(index+1)
                descriptions.append(description)
                inputs.append(input_data)
                expected_outputs.append(expected_output)
            
    # Create a dataframe
    data = {
        "User Story":user_story,
        "Description": descriptions,
        "Input": inputs,
        "Expected Output": expected_outputs
    }

:      Record ID: 12345  Expected Output:      Record 12345 is deleted from the database.
:      Record ID: 12345  Expected Output:      Record 12345 is deleted from the database.
:      Record IDs: 12345, 67890, 98765  Expected Output:      Records 12345, 67890, and 98765 are deleted from the database.
:      Record IDs: 12345, 67890, 98765  Expected Output:      Records 12345, 67890, and 98765 are deleted from the database.
:      Record ID: 12345      Child Record IDs: 54321, 65432  Expected Output:      Record 12345 and its child records (54321 and 65432) are deleted from the database.
:      Record ID: 12345      Child Record IDs: 54321, 65432  Expected Output:      Record 12345 and its child records (54321 and 65432) are deleted from the database.
:      Record ID: 12345      Referencing Record ID: 67890  Expected Output:      Record 12345 cannot be deleted because it is referenced by record 67890.      An error message is generated.
:      Record ID: 12345      Referencing Recor

IndexError: list index out of range

In [23]:
d = pd.DataFrame(data)

In [44]:
data

{'User Story': ' I want my domain models to be indexed properly, so that I can get validation results back in a reasonable amount of time.',
 'Description': [': Validate the indexing of a simple domain model with a single property.',
  ': Validate the indexing of a domain model with a complex property.',
  ': Validate the indexing of a domain model with a foreign key.',
  ': Validate the indexing of a domain model with a composite key.',
  ': Validate the indexing of a domain model with multiple indexes.'],
 'Input': [': Domain model: class Person { public string Name { get; set; } } Index definition: `IndexBuilder.CreateFor().On(p => p.Name)``',
  ': Domain model: class Address { public string Street { get; set; } public string City { get; set; } } Index definition: IndexBuilder.CreateFor<Address>().On(a => a.Street).On(a => a.City)',
  ': Domain model: class Order { public int CustomerId { get; set; } } Index definition: IndexBuilder.CreateFor<Order>().On(o => o.CustomerId)',
  ': Do